In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
openai_api_key = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Tool_1 Tavily

In [4]:
search = TavilySearchResults(max_results=2)

In [5]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-25',
  'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area. ... Tuesday 06/25 ...'},
 {'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719294571, 'localtime': '2024-06-24 22:49'}, 'current': {'last_updated_epoch': 1719294300, 'last_updated': '2024-06-24 22:45', 'temp_c': 12.1, 'temp_f': 53.7, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 7.8, 'wind_kph': 12.6, 'wind_degree': 246, 'wind_dir': 'WSW', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'clo

# Tool_2 Retriever

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [7]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

# Tool_3 Stock_Market

In [26]:
@tool
def identify_exchange_market(url: str) -> str:
    """
    This function checks the input URL and returns the corresponding exchange name.
    Parameters:
    url (str): The URL to be checked.
    Returns:
    str: The name of the exchange in Chinese.
    """
    if "sse.com.cn" in url:
        return "上海交易所"
    elif "szse.cn" in url:
        return "深圳交易所"
    elif "sgx.com" in url:
        return "新加坡交易所"
    elif "hkexnews.hk" in url:
        return "香港交易所"
    else:
        return "no match is found"  # Return "Unknown Exchange" if no match is found

# Collect 3 Tools

In [23]:
tools = [search, retriever_tool, identify_exchange_market]

In [10]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [27]:
model_with_tools = llm.bind_tools(tools)

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")]) #Irrlevant content

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [24]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])# Relevant content

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Glozi2YKSMG3EgcqyqbOTXdQ'}]


In [28]:
response = model_with_tools.invoke([HumanMessage(content="This is an report(http://www.sse.com.cn/disclosure/listedinfo/announcement/c/new/2024-05-25/600885_20240525_X3DI.pdf) issued in an exchange market\
help me identify the exchange market\
MUST FOLLOW:\
only return the the exchange market")])# Relevant content

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'identify_exchange_market', 'args': {'url': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/new/2024-05-25/600885_20240525_X3DI.pdf'}, 'id': 'call_uA3jPiBHWFL8TzMsh057i0zE'}]


# Create Agent

In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

# Run the agent

In [17]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for building production-grade LLM applications that can help with testing. It allows you to closely monitor and evaluate your application, ensuring you can ship quickly and with confidence. LangSmith works independently, and you can follow these steps to get started:\n\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on "Create API Key".\n\n3. Set up your environment:\n   - Export the following environment variables:\n     ```\n     export LANGCHAIN_TRACING_V2=true\n     export LANGCHAIN_API_KEY=<your-api-key>\n     export OPENAI_API_KEY=<your-openai-api-key> (optional)\n     ```\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces for evaluation.\n\nBy following these steps and using LangSmith, you can improve your te

In [19]:
agent_executor.invoke({"input": "whats the weather in sf?"})

{'input': 'whats the weather in sf?',
 'output': 'I found some information on the weather in San Francisco. You can check the [San Francisco Weather History for the Previous 24 Hours](https://www.timeanddate.com/weather/usa/san-francisco/historic) or view the [Extended Weather Forecast for June 2024](https://world-weather.info/forecast/usa/san_francisco/june-2024/) for details.'}

In [29]:
agent_executor.invoke({"input": "This is an report(http://www.sse.com.cn/disclosure/listedinfo/announcement/c/new/2024-05-25/600885_20240525_X3DI.pdf) issued by an exchange market\
help me identify the exchange market\
MUST FOLLOW:\
only return the the exchange market"})

{'input': 'This is an report(http://www.sse.com.cn/disclosure/listedinfo/announcement/c/new/2024-05-25/600885_20240525_X3DI.pdf) issued in an exchange markethelp me identify the exchange marketMUST FOLLOW:only return the the exchange market',
 'output': 'The exchange market mentioned in the report is the Shanghai Stock Exchange (SSE).'}